<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/WRDS_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wrds

     |████████████████████████████████| 3.4 MB 39.2 MB/s 


In [ ]:
import wrds
db = wrds.Connection(wrds_username = "limsc")
db.create_pgpass_file()

Enter your WRDS username [limsc]:limsc
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


In [ ]:
import pandas as pd
tickers = pd.read_csv("Russell 3000 Tickers.csv", dtype = str)
tickers = tickers["Ticker"].tolist()
len(tickers)

2958

In [ ]:
db.list_libraries()
libs = ["ravenpack_common", "ravenpack_dj", "ravenpack_pr"]

In [ ]:
company_mapping = db.get_table(library = "ravenpack_common",
                               table = "wrds_company_mapping")

In [ ]:
company_mapping["ticker"] = company_mapping["ticker"].astype(str)
company_mapping["country"] = company_mapping["isin"].str[:2]
company_mapping = company_mapping[company_mapping["ticker"].isin(tickers) &
                                  company_mapping["country"].isin(["US"])]

ravenpack_entityIDs = set(company_mapping["rp_entity_id"])

In [ ]:
rp_dj_tables = db.list_tables(library = "ravenpack_dj")[18:22]
rp_dj_tables

['dj_equities_2018',
 'dj_equities_2019',
 'dj_equities_2020',
 'dj_equities_2021']

In [ ]:
"""
LONG EXECUTION TIME
"""

yr = 2018

for i in range(4):
  raw = db.get_table(library = "ravenpack_dj",
                     table = rp_dj_tables[i],
                     columns = ["rp_entity_id", "entity_name", "category", "relevance", "ess", "rpna_date_utc", "timestamp_utc"])
  
  clean = raw[raw["rp_entity_id"].isin(ravenpack_entityIDs) &
              raw["ess"] > 0]
  
  name = "dj_rp_" + str(yr) + ".csv"
  clean.to_csv(name)
  yr = yr + 1
  print(name)

dj_rp_2018.csv
dj_rp_2019.csv
dj_rp_2020.csv
dj_rp_2021.csv
